In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import xgboost as xgb
import numpy as np
import pandas as pd
from statistics import mean
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import xgboost as xgb


def date_trans(df):
    df['Policy Start Date']= pd.to_datetime(df['Policy Start Date'])
    df['Year'] = df['Policy Start Date'].dt.year
    df['Day'] = df['Policy Start Date'].dt.day
    df['Month'] = df['Policy Start Date'].dt.month
    df['DayOfWeek'] = df['Policy Start Date'].dt.dayofweek
    df.drop('Policy Start Date' , axis =1, inplace = True)
    return df


def feature_gen(df1: pd.DataFrame):
    # Convert columns to numeric, handling potential string values
    columns_to_convert = [
        "Insurance Duration",
        "Health Score", 
        "Credit Score", 
        "Previous Claims"
    ]

    df1[['Previous Claims','Insurance Duration']] = df1[['Previous Claims','Insurance Duration']] .replace('unknown', 5)
    
    for col in columns_to_convert:
        df1[col] = pd.to_numeric(df1[col], errors='coerce')
    
    # Feature generation
    df1["Vehicle_Age_Insurance_Duration"] = df1["Vehicle Age"] / df1["Insurance Duration"]
    
    df1["Annual_Income_Health_Score_Ratio"] = df1["Annual Income"] / df1["Health Score"] 
    df1["Annual_Income_Health_Score"] = df1["Annual Income"] * df1["Health Score"]
    
    df1["Annual_Income_Credit_Score_Ratio"] = df1["Annual Income"] / df1["Credit Score"]
    df1["Annual_Income_Credit_Score"] = df1["Annual Income"] * df1["Credit Score"]
    
    df1['Annual_Income_Previous_Claims'] = df1['Previous Claims'] * df1['Annual Income']
    
    df1['Policy Start Month Sin'] = np.sin(df1['Month'] / 12 * 2 * np.pi)
    df1['Policy Start Month Cos'] = np.cos(df1['Month'] / 12 * 2 * np.pi)
    df1['Policy Start Day Sin'] = np.sin(df1['Day'] / 31 * 2 * np.pi)
    df1['Policy Start Day Cos'] = np.cos(df1['Day'] / 31 * 2 * np.pi)
    df1['Policy Start DayOfWeek Sin'] = np.sin(df1['DayOfWeek'] / 7 * 2 * np.pi)
    df1['Policy Start DayOfWeek Cos'] = np.cos(df1['DayOfWeek'] / 7 * 2 * np.pi)

    
    for col in columns_to_convert:
        df1[col] = pd.Categorical(df1[col])
        
    return df1

import numpy as np

def rmsle(predicted, actual, epsilon=1e-10):
    # Ensure inputs are numpy arrays
    predicted = np.array(predicted)
    actual = np.array(actual)
    
    # Clip negative values to zero
    predicted = np.clip(predicted, 0, None)
    actual = np.clip(actual, 0, None)
    
    # Add epsilon to zero values to prevent log(0)
    predicted = np.where(predicted == 0, epsilon, predicted)
    actual = np.where(actual == 0, epsilon, actual)
    
    # Compute log-transformed values
    log_predicted = np.log1p(predicted)  # np.log1p is numerically more stable for small values
    log_actual = np.log1p(actual)
    
    # Calculate the squared differences
    log_diff = log_predicted - log_actual
    rmsle_value = np.sqrt(np.mean(np.square(log_diff)))  # Compute RMSLE
    
    return rmsle_value

In [9]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv',low_memory=False)
test_df = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv', low_memory=False).drop(columns='id')

train_1 = date_trans(train_df)
test_1 = date_trans(test_df)

train = feature_gen(train_1)
test= feature_gen(test_1)


In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class LabelEncoderTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # No fitting required for LabelEncoder

    def transform(self, X):
        encoder = LabelEncoder()
        return X.apply(encoder.fit_transform)

In [17]:
# Assume train and test are already loaded as DataFrames
for col in train.select_dtypes(include='object').columns:
    train[col] = train[col].astype('category')
for col in test.select_dtypes(include='object').columns:
    test[col] = test[col].astype('category')

folds = 5
xgb_models = []
xgb_oof = []
val_ll = []
val_index = []
val_predictions_1 = {}
val_actual_targets = {}
test_predictions_xgb_1 = np.zeros(len(test))
FOLDS = folds
counter = 1

X = train.drop(columns=["Premium Amount","id","kfold"])
y = train['Premium Amount']

# Define preprocessor for encoding and scaling
categorical_cols = X.select_dtypes(include=['category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', Pipeline(steps=[
            ('label_encoder', LabelEncoder())
        ]), categorical_cols)
    ],
    remainder='passthrough'
)

for fold in range(folds):
    print(f"Training fold {fold + 1}")

    # Split data into training and validation sets
    xtrain = train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    valid_ids = xvalid.id.values.tolist()
    ytrain = xtrain['Premium Amount']
    yvalid = xvalid['Premium Amount']
    val_actual_targets.update(zip(valid_ids, yvalid.values))

    # Drop unnecessary columns
    xtrain = xtrain.drop(['Premium Amount', 'kfold', 'id'], axis=1)
    xvalid = xvalid.drop(['Premium Amount', 'kfold', 'id'], axis=1)

    # Apply preprocessing to training and validation sets
    xtrain_preprocessed = preprocessor.fit_transform(xtrain)
    xvalid_preprocessed = preprocessor.transform(xvalid)
    X_test_preprocessed = preprocessor.transform(xtest)


    # Create DMatrix for train, validation, and test
    dtrain = xgb.DMatrix(data=xtrain_preprocessed, label=ytrain)
    dvalid = xgb.DMatrix(data=xvalid_preprocessed, label=yvalid)
    dtest = xgb.DMatrix(data=X_test_preprocessed)

    # Watchlist for evaluation
    watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

    # Parameters for XGBoost
    params = {
        'objective': 'reg:squarederror',
        'tree_method': 'hist',  # Use GPU for faster training
        'device': 'cuda',
        'eval_metric': 'rmsle',
        'max_delta_step':1,# Metric to evaluate
        'seed': 42,
        'learning_rate': 0.19337999955239743,
        'max_depth': 10,
        'min_child_weight': 6.619545760132077,
        'subsample': 0.6361292838431416,
        'colsample_bytree': 0.9711731770736601,
        'lambda': 8.002007565484153,
        'alpha': 9.12137999233302} 

    # Train the model
    model = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=10000,
        evals=watchlist,
        early_stopping_rounds=300,
        verbose_eval=0
    )

    # Predictions for validation set
    val_preds = model.predict(dvalid)
    val_predictions_1.update(zip(valid_ids, val_preds))

    # Calculate RMSLE
    val_score = rmsle(yvalid.values, val_preds)
    best_iter = model.best_iteration
    print(f"RMSLE: {val_score:.5f} and the best iteration is: {best_iter}")
    print("*" * 50)

    # Store validation score and model
    val_ll.append(val_score)
    xgb_models.append(model)

    # Predictions for the test set (average over folds)
    test_preds = model.predict(dtest) / folds
    test_predictions_xgb_1 += test_preds

# Save predictions
final_valid_predictions = pd.DataFrame.from_dict(val_predictions_1, orient="index").reset_index()
final_valid_predictions.columns = ["id", "xgb_prediction"]
final_valid_predictions['actual_target'] = final_valid_predictions['id'].map(val_actual_targets)
final_valid_predictions.to_csv("train_pred_xgb.csv", index=False)
print(f"Root Mean Square Log Loss : {np.mean(val_ll)}")


Training fold 1


TypeError: LabelEncoder.fit_transform() takes 2 positional arguments but 3 were given

In [ ]:
sub = pd.read_csv('playground-series-s4e12/sample_submission.csv')
sub['Premium Amount'] = test_predictions_xgb_1
sub.to_csv('submission.csv', index=False)
sub.head()